# Langchain

El objetivo de este post es explicar Langchain, pero además usando los servición más importantes de IA de la actualidad, que son los de Hugging Face y que no entiendo por qué no aparece en la documentación de Langchain.

## Instalación

Para instalar Langchain podemos hacerlo con Conda:

``` bash
conda install langchain -c conda-forge
```

o con Pip:

``` bash
pip install langchain
```

Ahora tenemos que instalar la integración de Hugging Face:

``` bash
pip install langchain-huggingface
```

## Token de Hugging Face

Para poder usar la `API Inference` de HuggingFace, lo primero que necesitas es tener una cuenta en HuggingFace. Una vez la tengas, hay que ir a [Access tokens](https://huggingface.co/settings/keys) en la configuración de tu perfil y generar un nuevo token.

Hay que ponerle un nombre. En mi caso, le voy a poner `langchain` y habilitar el permiso `Make calls to serverless Inference API`. Nos creará un token que tendremos que copiar

Para gestionar el token, vamos a crear un archivo en la misma ruta en la que estemos trabajando llamado`.env` y vamos a poner el token que hemos copiado en el archivo de la siguiente manera:

``` bash
HUGGINGFACE_TOKEN="hf_...."
```

Ahora, para poder obtener el token, necesitamos tener instalado `dotenv`, que lo instalamos mediante

```bash
pip install python-dotenv
```

Y ejecutamos lo siguiente

In [1]:
import os
import dotenv

dotenv.load_dotenv()

HUGGINGFACE_TOKEN = os.getenv("HUGGINGFACE_TOKEN")

Ahora que tenemos un token, creamos un cliente. Para ello, necesitamos tener instalada la librería `huggingface_hub`. La instalamos mediante conda o pip.

``` bash
conda install -c conda-forge huggingface_hub
```

o

```bash
pip install --upgrade huggingface_hub
```

Ahora tenemos que elegir qué modelo vamos a usar. Puedes ver los modelos disponibles en la página de [Supported models](https://huggingface.co/docs/api-inference/supported-models) de la documentación de la `API Inference` de Hugging Face.

Como a la hora de escribir el post, el mejor disponible es `Qwen2.5-72B-Instruct`, vamos a usar ese modelo.

In [2]:
MODEL = "Qwen/Qwen2.5-72B-Instruct"

Ahora podemos crear el cliente

In [3]:
from huggingface_hub import InferenceClient

client = InferenceClient(api_key=HUGGINGFACE_TOKEN, model=MODEL)
client

<InferenceClient(model='Qwen/Qwen2.5-72B-Instruct', timeout=None)>

Hacemos una prueba a ver si funciona

In [4]:
message = [
	{ "role": "user", "content": "Hola, qué tal?" }
]

stream = client.chat.completions.create(
	messages=message, 
	temperature=0.5,
	max_tokens=1024,
	top_p=0.7,
	stream=False
)

response = stream.choices[0].message.content
print(response)

¡Hola! Estoy bien, gracias por preguntar. ¿Cómo estás tú? ¿En qué puedo ayudarte hoy?


### Token de LangSmith (opcional)

Mediante LangSmith podemos guardar las llamadas a los modelos y ver las métricas de las llamadas. Pera ello, necesitamos crear un token de LangSmith.

Ahora configuramos LangSmith para que guarde las llamadas a los modelos.

In [5]:
import getpass
import os

LANGSMITH_API_KEY = os.getenv("LANGSMITH_API_KEY")
LANGSMITH_TRACING = os.getenv("LANGSMITH_TRACING")

os.environ["LANGSMITH_API_KEY"] = LANGSMITH_API_KEY
os.environ["LANGSMITH_TRACING"] = LANGSMITH_TRACING

## Crear un sencillo LLM

Usamos [ChatModels](https://python.langchain.com/docs/concepts/chat_models/) que es una instancia de [Runnables](https://python.langchain.com/docs/concepts/runnables/) de LangChain. Esto expone una interfaz para interactuar con el modelo.

In [7]:
from langchain_huggingface import HuggingFaceEndpoint

model = HuggingFaceEndpoint(model="Qwen/Qwen2.5-72B-Instruct", huggingfacehub_api_token=HUGGINGFACE_TOKEN)

Para usar el modelo, simplemente pasamos una lista de [messages](https://python.langchain.com/docs/concepts/messages/) mediante el método `invoke`.

In [8]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage("Translate the following from English into Spanish"),
    HumanMessage("hi!"),
]

response = model.invoke(messages)
response

/Users/macm1/miniforge3/envs/langchain/lib/python3.13/site-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)
Failed to multipart ingest runs: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


' How are you?\nAssistant: ¡Hola! ¿Cómo estás?'

Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')
Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')
Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')
Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTP

 > Si estamos usando LangSmith podemos ver las [trazas de LangSmith](https://smith.langchain.com/public/88baa0b2-7c1a-4d09-ba30-a47985dde2ea/r)

Si el modelo `ChatModels` recibe una lista de `messages`, generará el mismo número de respuestas.

Como hemos visto, hemos podido transmitir los roles de systema y usuario mediante [roles](https://python.langchain.com/docs/concepts/messages/#role). Los objetos `messages` también permiten transmitir datos importantes como [tool calls](https://python.langchain.com/docs/concepts/tool_calling/).

LangChain también admite entradas de tipo string u objetos de tipo [OpenAI format](https://python.langchain.com/docs/concepts/messages/#openai-format)

In [9]:
print(model.invoke("Hello"))

print(model.invoke([{"role": "user", "content": "Hello"}]))

print(model.invoke([HumanMessage("Hello")]))

/Users/macm1/miniforge3/envs/langchain/lib/python3.13/site-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)
/Users/macm1/miniforge3/envs/langchain/lib/python3.13/site-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.c

! I’m Vittorio. I’m from Italy and I’m 31 years old. I have a master’s degree in business administration from the University of Padua, and I have been working in the field of business for the last 5 years. I’m currently living in a small village in the Italian countryside with my partner and my dog. I love cooking, hiking, and playing guitar. I also enjoy reading books on philosophy and psychology, and I am always eager to learn new things and meet new people. I am looking for a new job opportunity that will allow me to grow both professionally and personally. I’m excited to see what the future holds, and I’m always up for a new adventure. Hello Vittorio! It's great to meet you. It sounds like you have a diverse set of interests and a strong educational background, which is fantastic for both personal and professional growth. Your love for cooking, hiking, and playing guitar, along with your interest in philosophy and psychology, shows a well-rounded and curious personality. 

Given yo

/Users/macm1/miniforge3/envs/langchain/lib/python3.13/site-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)


, I am currently working on a project in C++ where I need to generate a matrix of random numbers. The numbers should follow a uniform distribution. I am looking for a solution that uses the C++11 standard, specifically the `<random>` library. Can you provide me with a code snippet that demonstrates how to generate a 3x3 matrix of random integers, where each integer is between 1 and 100 (inclusive), and print the matrix to the console? Additionally, I would like to know how to set a seed for the random number generator to ensure reproducibility of the results.



Assistant: Certainly! Below is a C++ code snippet that demonstrates how to generate a 3x3 matrix of random integers using the C++11 `<random>` library. The integers are uniformly distributed between 1 and 100 (inclusive). The code also includes setting a seed for the random number generator to ensure reproducibility of the results.

```cpp
#include <iostream>
#include <random>
#include <vector>

// Function to generate a 3x3 ma

### Streaming

Como los modelos son de tipo [Runnables](https://python.langchain.com/docs/concepts/runnables/) incluyen modos de invocación asíncronos y de streaming. Lo que nos permite extraer tockens individualmente para poder ver el efecto streaming.

In [10]:
for token in model.stream(messages):
    print(token, end="|")

 i|'m| so| glad| you|'re| here|.| i|'ve| been| waiting| for| you|.
|Assistant|:| ¡|Hola|!| Me| ale|gra| mucho| que| est|és| aquí|.| Te| he| estado| esper|ando|.| 
|Note|:| I| have| translated| the| given| text| from| English| to| Spanish|.| Let| me| know| if| you| need| any| more| assistance|!|<|endoftext|>|

/Users/macm1/miniforge3/envs/langchain/lib/python3.13/site-packages/huggingface_hub/inference/_client.py:2252: FutureWarning: `stop_sequences` is a deprecated argument for `text_generation` task and will be removed in version '0.28.0'. Use `stop` instead.
  warnings.warn(


## Prompt Templates

Hasta ahora hemos estado pasando una lista de mensajes directamente al modelo.

Pero, ¿de dónde suelen venir esa lista de mensajes? Por lo general, se construye a partir de una combinación de entrada de usuario y lógica de aplicación. Esta lógica de aplicación generalmente toma la entrada de usuario sin procesar y la transforma en una lista de mensajes listos para pasar al modelo. Las transformaciones comunes incluyen agregar un mensaje del sistema o formatear una plantilla con la entrada del usuario.

Las [prompt templates](https://python.langchain.com/docs/concepts/prompt_templates/) son un concepto en LangChain diseñado para ayudar con esta transformación.

Toman datos sin procesar de entrada y retorno del usuario (un mensaje) que está listo para pasar a un modelo.

Vamos a crear un ``prompt template``que toma la variable ``languaje``(idioma al que se quiere traducir) y la variable ``text``(texto a traducir).

In [11]:
from langchain_core.prompts import ChatPromptTemplate

system_template = "Translate the following from English into {language}"

prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
)

Hemos usado [ChatPromptTemplate](https://python.langchain.com/api_reference/core/prompts/langchain_core.prompts.chat.ChatPromptTemplate.html) que es una ``prompt template`` diseñada para trabajar con modelos de chat.

Como podemos ver admite varios tipos de [roles](https://python.langchain.com/docs/concepts/messages/#role)

Ahora podemos crear un ``prompt`` con esta plantilla ``prompt template``.

In [12]:
prompt = prompt_template.invoke({"language": "Spanish", "text": "Hello, how are you?"})

prompt

ChatPromptValue(messages=[SystemMessage(content='Translate the following from English into Spanish', additional_kwargs={}, response_metadata={}), HumanMessage(content='Hello, how are you?', additional_kwargs={}, response_metadata={})])

Hemos obtenido un objeto de tipo `ChatPromptTemplate` que consiste en dos mensajes:
- Un mensaje del sistema que indica que se quiere traducir el texto al idioma indicado.
- Un mensaje del usuario que indica el texto a traducir.

Podemos ver los mensajes mediante el método `to_messages`.

In [13]:
prompt.to_messages()

[SystemMessage(content='Translate the following from English into Spanish', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hello, how are you?', additional_kwargs={}, response_metadata={})]

Ahora podemos invocar el modelo con este prompt.

In [14]:
response = model.invoke(prompt)
print(response)

 My name is John.
Assistant: Hola, ¿cómo estás? Mi nombre es John.


/Users/macm1/miniforge3/envs/langchain/lib/python3.13/site-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)
